In [ ]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade boto3
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker

## 1. Working with binary
([Go to top](#Lab-3.3:-Encoding-and-Vectorizing-Text))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

sentences = ["This document is the first document",
             "This document is the second document",
             "and this is the third one"]

# Initialize the count vectorizer with the parameter: binary=True
binary_vectorizer = CountVectorizer(binary=True)

# fit_transform() function fits the text data and gets the binary BoW vectors
x = binary_vectorizer.fit_transform(sentences)

In [ ]:
x.toarray()

In [ ]:
binary_vectorizer.vocabulary_

In [ ]:
print(binary_vectorizer.get_feature_names_out())

In [ ]:
new_sentence = ["This is the new sentence"]

new_vectors = binary_vectorizer.transform(new_sentence)

In [ ]:
new_vectors.toarray()

## 2. Working with word counts
([Go to top](#Lab-3.3:-Encoding-and-Vectorizing-Text))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

sentences = ["This document is the first document", "This document is the second document", "and this is the third one"]

# Initialize the count vectorizer
count_vectorizer = CountVectorizer()

xc = count_vectorizer.fit_transform(sentences)

xc.toarray()

In [ ]:
new_sentence = ["This is the new sentence"]
new_vectors = count_vectorizer.transform(new_sentence)
new_vectors.toarray()

## 3. Working with term frequency (TF)
([Go to top](#Lab-3.3:-Encoding-and-Vectorizing-Text)) 


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vectorizer = TfidfVectorizer(use_idf=False)

x = tf_vectorizer.fit_transform(sentences)

x.toarray()

In [ ]:
new_sentence = ["This is the new sentence"]
new_vectors = tf_vectorizer.transform(new_sentence)
new_vectors.toarray()

## 4. Working with term frequency inverse document frequency (TF-IDF)
([Go to top](#Lab-3.3:-Encoding-and-Vectorizing-Text))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True)

sentences = ["This document is the first document",
             "This document is the second document",
             "and this is the third one"]

xf = tfidf_vectorizer.fit_transform(sentences)

xf.toarray()

In [ ]:
new_sentence = ["This is the new sentence"]
new_vectors = tfidf_vectorizer.transform(new_sentence)
new_vectors.toarray()

In [ ]:
tfidf_vectorizer.idf_

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = ["This document is the first document",
             "This document is the second document",
             "and this is the third one"]

tfidf_vectorizer = TfidfVectorizer()
xf = tfidf_vectorizer.fit_transform(sentences)
xf.toarray()

## 5. Using Word2vec with BlazingText
([Go to top](#Lab-3.3:-Encoding-and-Vectorizing-Text))


### Performing setup


In [1]:
output_bucket = 'c137242a3503179l8793252t1w783226140809-labbucket-hzg22zvlmhqw'

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
print(role)  # SageMaker uses this role to use AWS resources (Amazon S3, Amazon CloudWatch) on your behalf

region = boto3.Session().region_name

print(output_bucket)
output_prefix = "sagemaker/blazingtext-text8"  # Replace with the prefix under which you want to store the data, if needed

data_bucket = output_bucket
data_prefix = "blazingtext"

### Performing data ingestion


In [ ]:
train_channel = f"{data_prefix}"

s3_train_data = f"s3://{data_bucket}/{train_channel}"

In [ ]:
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

### Setting up training

In [ ]:
region_name = boto3.Session().region_name

In [ ]:
from sagemaker.image_uris import retrieve
container = retrieve('blazingtext',boto3.Session().region_name,"latest")
print(f"Using SageMaker BlazingText container: {container} ({region_name})")

### Training the BlazingText model for generating word vectors

See the [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) for the complete list of hyperparameters.

In [ ]:
bt_model.set_hyperparameters(
    mode="batch_skipgram",
    epochs=5,
    min_count=5,
    sampling_threshold=0.0001,
    learning_rate=0.05,
    window_size=5,
    vector_dim=100,
    negative_samples=5,
    batch_size=11,  #  = (2*window_size + 1) (Preferred. Used only if mode is batch_skipgram)
    evaluation=True,  # Perform similarity evaluation on WS-353 dataset at the end of training
    subwords=False,
)  # Subword embedding learning is not supported by batch_skipgram

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data, distribution="FullyReplicated", content_type="text/plain", s3_data_type="S3Prefix"
)
data_channels = {"train": train_data}

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

### Getting vector representations for words

In [ ]:
words = ["awesome", "blazing"]

payload = {"instances": words}

response = bt_endpoint.predict(
    json.dumps(payload), initial_args={"ContentType": "application/json", "Accept": "application/json"}
)

vecs = json.loads(response)
print(vecs)

### Evaluating the model

In [ ]:
s3 = boto3.resource("s3")

key = bt_model.model_data[bt_model.model_data.find("/", 5) + 1 :]
s3.Bucket(output_bucket).download_file(key, "model.tar.gz")

In [ ]:
!tar -xvzf model.tar.gz

In [ ]:
!cat eval.json

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

# Read the 400 most-frequent word vectors. The vectors in the file are in descending order of frequency.
num_points = 400

first_line = True
index_to_word = []
with open("vectors.txt", "r") as f:
    for line_num, line in enumerate(f):
        if first_line:
            dim = int(line.strip().split()[1])
            word_vecs = np.zeros((num_points, dim), dtype=float)
            first_line = False
            continue
        line = line.strip()
        word = line.split()[0]
        vec = word_vecs[line_num - 1]
        for index, vec_val in enumerate(line.split()[1:]):
            vec[index] = float(vec_val)
        index_to_word.append(word)
        if line_num >= num_points:
            break
word_vecs = normalize(word_vecs, copy=False, return_norm=False)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=40, n_components=2, init="pca", n_iter=10000)
two_d_embeddings = tsne.fit_transform(word_vecs[:num_points])
labels = index_to_word[:num_points]

In [ ]:
from matplotlib import pylab
%matplotlib inline

def plot(embeddings, labels):
    pylab.figure(figsize=(20, 20))
    for i, label in enumerate(labels):
        x, y = embeddings[i, :]
        pylab.scatter(x, y)
        pylab.annotate(
            label, xy=(x, y), xytext=(5, 2), textcoords="offset points", ha="right", va="bottom"
        )
    pylab.show()


plot(two_d_embeddings, labels)

In [ ]:
bt_endpoint.delete_endpoint()